### Q1

In [1]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

In [2]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get the weather data for a city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "City as a text field to look up the weather."
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

In [3]:
!wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

--2025-07-14 23:56:25--  https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3485 (3.4K) [text/plain]
Saving to: ‘chat_assistant.py.2’

chat_assistant.py.2 100%[===================>]   3.40K  --.-KB/s    in 0s      

2025-07-14 23:56:25 (39.0 MB/s) - ‘chat_assistant.py.2’ saved [3485/3485]



In [4]:
from openai import OpenAI
client = OpenAI()

In [5]:
import chat_assistant

tools = chat_assistant.Tools()
tools.add_tool(get_weather, get_weather_tool)

tools.get_tools()

developer_prompt = """
You're a weather forecaster. 
You're given a city for which you need to provide the temperature.

Use known_weather_data if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [6]:
chat.run()

You: What is the weather in Mumbai?


You: The weather in Mumbai India 


You: stop


Chat ended.


### Q2

In [7]:
def set_weather(city, temp) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [8]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Set the weather data for a city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "City as a text field to set the weather."
            },
            "temp": {
                "type": "number",
                "description": "Temperature as a float field to set the weather."
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

In [9]:
tools.add_tool(set_weather, set_weather_tool)
tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get the weather data for a city',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'City as a text field to look up the weather.'}},
   'required': ['city'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'set_weather',
  'description': 'Set the weather data for a city',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'City as a text field to set the weather.'},
    'temp': {'type': 'number',
     'description': 'Temperature as a float field to set the weather.'}},
   'required': ['city', 'temp'],
   'additionalProperties': False}}]

In [10]:
chat.run()

You: Update the weather in Mumbai, India as 26 Deg


You: stop


Chat ended.


In [16]:
#!pip install fastmcp

In [17]:
pip show fastmcp

/usr/local/python/3.12.1/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=5836) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


Name: fastmcp
Version: 2.10.5
Summary: The fast, Pythonic way to build MCP servers and clients.
Home-page: https://gofastmcp.com
Author: Jeremiah Lowin
Author-email: 
License-Expression: Apache-2.0
Location: /usr/local/python/3.12.1/lib/python3.12/site-packages
Requires: authlib, cyclopts, exceptiongroup, httpx, mcp, openapi-pydantic, pydantic, pyperclip, python-dotenv, rich
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [18]:
# weather_server.py
from fastmcp import FastMCP

mcp = FastMCP("Demo 🚀")

@mcp.tool
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)


def set_weather(city: str, temp: float) -> None:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [14]:
# mcp.run() causes runtime error
# await mcp.run_async() causes attribute error
await mcp.run_async(transport="http", port=8080)

╭─ FastMCP 2.0 ──────────────────────────────────────────────────────────────╮
│                                                                            │
│        _ __ ___ ______           __  __  _____________    ____    ____     │
│       _ __ ___ / ____/___ ______/ /_/  |/  / ____/ __ \  |___ \  / __ \    │
│      _ __ ___ / /_  / __ `/ ___/ __/ /|_/ / /   / /_/ /  ___/ / / / / /    │
│     _ __ ___ / __/ / /_/ (__  ) /_/ /  / / /___/ ____/  /  __/_/ /_/ /     │
│    _ __ ___ /_/    \__,_/____/\__/_/  /_/\____/_/      /_____(_)____/      │
│                                                                            │
│                                                                            │
│                                                                            │
│    🖥️  Server name:     Demo 🚀                                             │
│    📦 Transport:       Streamable-HTTP                                     │
│    🔗 Server URL:      http://127.0.0.1:8080/mcp/                          │
│                                                                            │
│    📚 Docs:            https://gofastmcp.com                               │
│    🚀 Deploy:          https://fastmcp.cloud                               │
│                                                                            │
│    🏎️  FastMCP version: 2.10.5                                              │
│    🤝 MCP version:     1.11.0                                              │
│                                                                            │
╰────────────────────────────────────────────────────────────────────────────╯

[07/14/25 23:58:55] INFO     Starting MCP server 'Demo 🚀' with transport 'http' on                  ]8;id=435800;file:///usr/local/python/3.12.1/lib/python3.12/site-packages/fastmcp/server/server.py\server.py]8;;\:]8;id=82774;file:///usr/local/python/3.12.1/lib/python3.12/site-packages/fastmcp/server/server.py#1448\1448]8;;\
                             http://127.0.0.1:8080/mcp/                                                            

INFO:     Started server process [5836]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8080 (Press CTRL+C to quit)
INFO:     Shutting down
ERROR:    Cancel 0 running task(s), timeout graceful shutdown exceeded
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [5836]
